In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Generate a small example dataset
np.random.seed(42)
dates = pd.date_range(start='2020-01-01', end='2022-12-31', freq='D')
n = len(dates)

# Generate synthetic data
data = pd.DataFrame({
    'date': dates,
    'TAVG': 20 + 5 * np.sin(2 * np.pi * (np.arange(n) / 365)) + np.random.normal(0, 2, n),  # Seasonal + noise
    'PRCP': np.random.gamma(2, 1, n),  # Random precipitation data
    'streamflow': 100 + 10 * np.sin(2 * np.pi * (np.arange(n) / 365)) + np.random.normal(0, 5, n)  # Seasonal + noise
})

# Calculate cumulative precipitation features
data['PREC_year'] = data.groupby(data['date'].dt.year)['PRCP'].cumsum()
data['PREC_month'] = data.groupby(data['date'].dt.month)['PRCP'].transform('sum')

# Train-test split (80-20)
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

# Define features and target
features = ['TAVG', 'PRCP', 'PREC_year']
target = 'streamflow'

X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Define SARIMAX model
model = SARIMAX(
    y_train,
    exog=X_train,
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 365),
    enforce_stationarity=False,
    enforce_invertibility=False
)

# Fit the model
model_fit = model.fit(disp=False)

# Forecast
y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, exog=X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(test_data['date'], y_test, label='Actual', color='blue')
plt.plot(test_data['date'], y_pred, label='Predicted', color='red')
plt.title(f'SARIMAX Example - RMSE: {rmse:.2f}')
plt.legend()
plt.show()

# Print results
rmse

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Simple synthetic data
data = pd.Series(100 + 10 * np.sin(np.linspace(0, 2 * np.pi, 365)) + np.random.normal(0, 5, 365))

# Train SARIMAX
model = SARIMAX(data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
result = model.fit(disp=False)
print(result.summary())

                                     SARIMAX Results                                     
Dep. Variable:                                 y   No. Observations:                  365
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 7)   Log Likelihood               -1102.953
Date:                           Tue, 26 Nov 2024   AIC                           2215.906
Time:                                   22:13:16   BIC                           2235.294
Sample:                                        0   HQIC                          2223.617
                                           - 365                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0636      0.063      1.014      0.310      -0.059       0.186
ma.L1         -0.8885      0.030    -30.014